In [ ]:
!pip install datasets transformers
!pip install accelerate
!pip install optimum
# !pip install datasets==2.15.0
!pip install wandb

In [ ]:
import os
import time
import datetime

import pandas as pd
import seaborn as sns
import numpy as np
import random

import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
torch.manual_seed(42)

from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import AutoModelForCausalLM

from transformers import Trainer, TrainingArguments


In [ ]:
model_checkpoint = "distilbert-base-uncased"


In [ ]:
from datasets import load_dataset
datasets = load_dataset('wikitext', 'wikitext-2-raw-v1')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
datasets['train'] = datasets['train'].select([i for i in range(10000)])


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [ ]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

In [ ]:
block_size = 512


In [ ]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/4358 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/10000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/3760 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

seq = 128,normal: batch = 32 , 4.3gb  time = 05:46 better: 4.0gb time = 05:44

seq = 128,normal: batch = 64 , 7.2gb   time 04:43 better: 6.6gb time = 04:48

seq = 128,normal: batch = 128, 14.0gb  time 4:23 better: 12.8gb  time == 4:23 (same)  

seq = 256, batch = 32 , 7.8gb  04:51 better 6.6gb  time 04:58

seq = 512, batch = 32 thì oom better 12.8gb time = 04:48   

seq = 512, batch = 16 time = 05:00, 9.1 better 6.6gb time = 04:53

seq 512, batch = 8 time = 06:30 5.2 better 4gb  time = 06:20


In [ ]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-wikitext2",
    evaluation_strategy = "steps",
    logging_steps = 20,
    eval_steps = 50,
    learning_rate=2e-5,
    weight_decay=0.01,
    report_to='none',
    # push_to_hub=True,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
)

In [ ]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    data_collator=data_collator,

)

512-8

In [ ]:
train_status= trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
50,2.311200,2.030740
100,2.190000,1.950243
150,2.163100,1.966640
200,2.083900,1.919730
250,2.087700,1.895792
300,2.018400,1.904611
350,2.031400,1.924766
400,2.017400,1.925428
450,2.035500,1.898607


In [ ]:
train_status

TrainOutput(global_step=468, training_loss=2.1119217628087754, metrics={'train_runtime': 392.8974, 'train_samples_per_second': 9.491, 'train_steps_per_second': 1.191, 'total_flos': 494320551081984.0, 'train_loss': 2.1119217628087754, 'epoch': 3.0})

512-16

In [ ]:
train_status= trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
100,2.150800,1.942309
200,2.053500,1.917114


256-32

In [ ]:
train_status= trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
50,2.249400,2.041987
100,2.136700,2.000196
150,2.157700,1.954933
200,2.084100,1.982292


128-32

In [ ]:
train_status= trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
50,2.460900,2.243208
100,2.317500,2.143974
150,2.227700,2.117158
200,2.233200,2.107711
250,2.234000,2.099533
300,2.168000,2.091412
350,2.123400,2.067453
400,2.175200,2.050605
450,2.264500,2.067394


128-64

In [ ]:
train_status= trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
50,2.390800,2.186466
100,2.245600,2.122296
150,2.284300,2.095545
200,2.172300,2.099282


In [ ]:
train_status= trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
50,2.370200,2.151389
100,2.251800,2.133358


In [ ]:
lm_datasets.save_to_disk("/content/lm_datasets_512")


Saving the dataset (0/1 shards):   0%|          | 0/546 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1243 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/479 [00:00<?, ? examples/s]

In [ ]:
import wandb
wandb.init(project="distilbert_wiki",name="Normal" )


eval/loss,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▁█
train/global_step,▁▁█
train/learning_rate,█▁
train/loss,█▁
eval/loss,2.39208
eval/runtime,11.7731
eval/samples_per_second,163.169


In [ ]:
train_status= trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
50,2.216100,2.008681
100,2.107100,1.977858
150,2.120300,1.910020
200,2.087500,1.904253


In [ ]:
import datasets
lm_datasets = datasets.load_from_disk("/content/lm_datasets_512")


In [ ]:
model_checkpoint = "distilbert-base-uncased"
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from transformers import AutoModelForMaskedLM
from optimum.bettertransformer import BetterTransformer
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)
model = BetterTransformer.transform(model)

The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.


In [ ]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [ ]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    f"{model_name}-finetuned-wikitext2",
    evaluation_strategy = "steps",
    logging_steps = 20,
    eval_steps = 50,
    learning_rate=2e-5,
    weight_decay=0.01,
    report_to='none',
    # push_to_hub=True,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    save_strategy = 'no'

)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    data_collator=data_collator,

)

512-32

In [ ]:
with torch.backends.cuda.sdp_kernel(
                    enable_flash=True, enable_math=False, enable_mem_efficient=True
                ):
  train_status_v2= trainer.train()

Step,Training Loss,Validation Loss
100,2.246100,1.975067


In [ ]:
train_status_v2

TrainOutput(global_step=117, training_loss=2.3298335279154982, metrics={'train_runtime': 290.6522, 'train_samples_per_second': 12.83, 'train_steps_per_second': 0.403, 'total_flos': 494320551081984.0, 'train_loss': 2.3298335279154982, 'epoch': 3.0})

512-16

In [ ]:
with torch.backends.cuda.sdp_kernel(
                    enable_flash=True, enable_math=False, enable_mem_efficient=True
                ):
  train_status_v2= trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
100,2.269700,1.959869
200,2.175200,1.935983


/usr/local/lib/python3.10/dist-packages/optimum/bettertransformer/models/encoder_models.py:868: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  hidden_states = torch._nested_tensor_from_mask(hidden_states, attn_mask)


In [ ]:
train_status_v2

TrainOutput(global_step=234, training_loss=2.2766954715435324, metrics={'train_runtime': 295.9105, 'train_samples_per_second': 12.602, 'train_steps_per_second': 0.791, 'total_flos': 494320551081984.0, 'train_loss': 2.2766954715435324, 'epoch': 3.0})

512-8

In [ ]:
with torch.backends.cuda.sdp_kernel(
                    enable_flash=True, enable_math=False, enable_mem_efficient=True
                ):
  train_status_v2= trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
50,2.456300,2.050755
100,2.300600,1.965805
150,2.281700,1.984574
200,2.198200,1.935816
250,2.213200,1.916167
300,2.149600,1.920027
350,2.142700,1.944777
400,2.143300,1.945200
450,2.161800,1.915836


/usr/local/lib/python3.10/dist-packages/optimum/bettertransformer/models/encoder_models.py:868: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  hidden_states = torch._nested_tensor_from_mask(hidden_states, attn_mask)


In [ ]:
train_status_v2

TrainOutput(global_step=468, training_loss=2.237732699793628, metrics={'train_runtime': 382.4925, 'train_samples_per_second': 9.749, 'train_steps_per_second': 1.224, 'total_flos': 494320551081984.0, 'train_loss': 2.237732699793628, 'epoch': 3.0})

256-32

In [ ]:
with torch.backends.cuda.sdp_kernel(
                    enable_flash=True, enable_math=False, enable_mem_efficient=True
                ):
  train_status_v2= trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
50,2.361400,2.059870
100,2.245300,2.015956
150,2.285200,1.970861
200,2.201200,1.999580


/usr/local/lib/python3.10/dist-packages/optimum/bettertransformer/models/encoder_models.py:868: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  hidden_states = torch._nested_tensor_from_mask(hidden_states, attn_mask)


In [ ]:
train_status_v2

TrainOutput(global_step=234, training_loss=2.2895310312254815, metrics={'train_runtime': 301.8883, 'train_samples_per_second': 24.764, 'train_steps_per_second': 0.775, 'total_flos': 495513601486848.0, 'train_loss': 2.2895310312254815, 'epoch': 3.0})

128-32

In [ ]:
with torch.backends.cuda.sdp_kernel(
                    enable_flash=True, enable_math=False, enable_mem_efficient=True
                ):
  train_status_v2= trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
50,2.592700,2.259719
100,2.435700,2.157389
150,2.350800,2.134360
200,2.340300,2.124124
250,2.361300,2.118548
300,2.274400,2.109758
350,2.245400,2.083834
400,2.286500,2.065627
450,2.390500,2.083140


/usr/local/lib/python3.10/dist-packages/optimum/bettertransformer/models/encoder_models.py:868: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  hidden_states = torch._nested_tensor_from_mask(hidden_states, attn_mask)


128-64

In [ ]:
with torch.backends.cuda.sdp_kernel(
                    enable_flash=True, enable_math=False, enable_mem_efficient=True
                ):
  train_status_v2= trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
50,2.520100,2.221107
100,2.354400,2.137901
150,2.397800,2.102857
200,2.284400,2.121587


/usr/local/lib/python3.10/dist-packages/optimum/bettertransformer/models/encoder_models.py:868: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at ../aten/src/ATen/NestedTensorImpl.cpp:178.)
  hidden_states = torch._nested_tensor_from_mask(hidden_states, attn_mask)


128 - 128

In [ ]:
with torch.backends.cuda.sdp_kernel(
                    enable_flash=True, enable_math=False, enable_mem_efficient=True
                ):
  train_status_v2= trainer.train()

Step,Training Loss,Validation Loss
50,2.495100,2.169735
100,2.364500,2.151352


In [ ]:
import wandb
wandb.init(project="distilbert_wiki",name="BetterTransformer" )


train/epoch,▁
train/global_step,▁
train/learning_rate,▁
train/loss,▁
train/epoch,0.8
train/global_step,500
train/learning_rate,1e-05
train/loss,2.4437
